<style>
    #rendered_cells { display:block !important; width:75vw; margin:auto }
    #Covid-19-Dashboard { font-size:2.5rem }
    h2 { padding: 1.2rem 0 1rem 0 !important }
    .alert { margin-top: 2rem }
</style>

[DIY Covid-19 Dashboard Kit](https://github.com/Ella-Natasha/Covid-dashboard) (C) Ellen Baker Price, 2020. *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and [data](https://www.ons.gov.uk/visualisations/dvc845/poppyramids/pyramids/datadownload.xlsx) published by the [Office for National Statistics](https://www.ons.gov.uk/)*

# Covid-19 Dashboard

This dashboard presents coronavirus pandemic data gathered from Public Heath England. The dashboard explores cumulative cases, hospitalisations and deaths in all countries in the UK, as well as cumulative cases by age and gender. For direct comparison of data between countries, all cumulative data show is a percentage of the population of each country. Population data has been gathered from the Office for National Statistics and uses the latest population data gathered in 2019. 

In [1]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API
import io
import requests
import time
import os
from ipywidgets import Layout, Button, Box, VBox

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
def get_population_data(year, skip_rows, skip_footer):
    """ Params: year - year as str, skiprows - , skip_footer - 
    Creates dataframe containing population data for each country in the UK by age and gender
    from Office of National Statistics.
    Col names: eg. m_19_4 means population of male 4 year olds in 2019.
    2019 data sheet is the lastest data until mid 2021."""

    excel_url="https://www.ons.gov.uk/visualisations/dvc845/poppyramids/pyramids/datadownload.xlsx"
    content=requests.get(excel_url).content
    xl_df = pd.read_excel(
        content,
        sheet_name=year,
        skiprows=skip_rows,
        skipfooter=skip_footer
    )
    return xl_df
    
def wrangle_population_data(data):
    """ Params: data - pd dataframe
    Creates pkl file containing population data for each country in the UK by age and gender
    from Office of National Statistics.
    Col names: eg. m_19_4 means population of male 4 year olds in 2019.
    2019 data sheet is the lastest data until mid 2021."""
    #Wrangle data and create dataframe
    xl_df.columns = xl_df.iloc[0]
    xl_df.drop(xl_df.index[0], inplace=True)
    xl_df.set_index("geogname", inplace=True)
    xl_df.index = xl_df.index.str.title()
    xl_df.drop(['variable','geogcode'], axis=1, inplace=True)

    ages = [f"{sex}_{i}_to_{i+4}" for i in range(0, 90, 5) for sex in ['m', 'f']]
    [ages.append(f"{sex}_{name}") for name in ['90', 'al'] for sex in ['m', 'f']]

    pop_by_age_df = pd.DataFrame(index=xl_df.index, columns=ages)
    pop_by_age_df.fillna(0.0, inplace=True)

    for sex in ['m', 'f']:
        for i in range(0, 90, 5):
            new_col_name = f"{sex}_{i}_to_{i+4}"
            for num in range(0,5):
                col_name = f"{sex}_19_{i + num}";
                pop_by_age_df[new_col_name] += xl_df[col_name]

        for name in ['90', 'al']:
            new_col_name = f"{sex}_{name}";
            col_name = f"{sex}_19_{name}";
            pop_by_age_df[new_col_name] += xl_df[col_name]

    pop_by_age_df['total'] = pop_by_age_df['m_al'] + pop_by_age_df['f_al']
    pop_by_age_df.rename(columns={'m_90': 'm_90+', 'f_90': 'f_90+'}, inplace=True)
    pop_by_age_df.to_pickle("popAge.pkl")

In [4]:
# Load all data

# Gets population data for each country in the UK by age and gender from Office of National Statistics. 
# Col names: eg. m_0_to_4 means population of male 0-4 year olds in 2019.
# 2019 data sheet is the lastest data until mid 2021.
# Data will be updated mid 2021 so need to remake pkl file using get_population_data()
# Current file made from get_population_data('2019', range(2,423), 2)
pop_by_age_df = pd.read_pickle("popAge.pkl")

# Load JSON file containing data on Cases, Hospitalisations and Deaths
with open("caseDeathHospitalisations.json", "rt") as INFILE:
    cdhjson=json.load(INFILE);

In [5]:
countries = ['England','Northern Ireland','Scotland','Wales'];
m_f = ['m', 'f']

In [6]:
# Create ages lists
ages = [];
ages_m_f = [];

for i in range(0, 90, 5):
    ages.append(f"{i}_to_{i+4}");
    for sex in m_f:
        ages_m_f.append(f"{sex}_{i}_to_{i+4}");
ages.append('90+')

for name in ['90+', 'al']:
    for sex in m_f:
        ages_m_f.append(f"{sex}_{name}")

ages_list = ages_m_f[: len(ages_m_f) - 2]
ages_list.insert(0, "Date")
ages_list.insert(1, "Country")
text_ages = [age.replace('_', ' ') for age in ages]

In [7]:
# Data frame creation functions

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def create_country_dfs(df, country):
    """ Parameters: df - maindataframe from JSON or API call, 
    country - counrty name as str as it apears in main dataframe.
    Returns: single dataframe.
    Creates a new single country dataframe from the main dataframe.
    Then sets index as date."""
    country_df = df[df['Country'] == country]
    country_df.set_index("Date", inplace=True)
    return country_df;

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. 
    Returns: dictionary of dataframes.
    Wrangles the data, creates the main dataframe and fills it in. 
    Then splits main dataframe into country dataframes"""
    
    data = rawdata['data'];
    
    # Create main data frame
    df = pd.DataFrame(index=range(len(data)), columns=[
        'Date',
        'Country',
        'Cumulative Cases',
        'Cumulative Cases % of Population',
        'Daily Cases',
        'Daily Cases % of Population',
        'Cumulative Hospitalisations',
        'Cumulative Hospitalisations % of Population',
        'Daily Hospitalisations',
        'Daily Hospitalisations % of Population',
        'Cumulative Deaths',
        'Cumulative Deaths % of Population',
        'Daily Deaths',
        'Daily Deaths % of Population',
    ])
    
    # Create cases by ages data frame
    age_df = pd.DataFrame(index=range(len(data)), columns=ages_list)
    
    count = 0;
    for entry in data:
        
        # Date and country columns for df and age_df
        for key, val in {'Date': 'date','Country': 'name'}.items():
            if pd.isna(df.loc[count, key]):
                df.loc[count, key] = age_df.loc[count, key] = parse_date(entry[val]) if key=='Date' else entry[val];
        
        for dataType in ['cases', 'hospitalisations', 'deaths']:
            dataTypeData = entry[dataType];
            
            #Populate df with values
            for timeFrame in ['daily', 'cumulative']:
                # Raw numbers columns
                colName = f"{timeFrame} {dataType}".title();
                if pd.isna(df.loc[count, colName]):
                    value = float(dataTypeData[timeFrame]) if dataTypeData[timeFrame]!= None else 0.0
                    df.loc[count, colName] = value
                    
                # % of population columns
                colNamePercent = f"{colName} % of Population"
                if pd.isna(df.loc[count, colNamePercent]):
                    precentValue = (df.loc[count, colName] / pop_by_age_df.loc[entry['name'], 'total']) * 100;
                    df.loc[count, colNamePercent] = precentValue;
                    
            # Populate age_df with values  
            if dataType == 'cases':
                for sex in ['males', 'females']:
                    age_data = dataTypeData[sex]
                    if age_data != None:
                        for data in age_data:
                            colName = f"{sex[0]}_{data['age']}";
                            if pd.isna(age_df.loc[count, colName]):
                                value = float(data['value']) if data['value']!= None else 0.0;
                                percent = (value / pop_by_age_df.loc[entry['name'], colName]) * 100;
                                age_df.loc[count, colName] = percent;
             
        count+=1;
        
    df.fillna(0.0, inplace=True)
    age_df.fillna(0.0, inplace=True)
    
    #Create country data frames
    dataframes = {}
    for country in countries:
        dataframes[country] = create_country_dfs(df, country)
        dataframes[f"{country}_ages"] = create_country_dfs(age_df, country)

    return dataframes;

dataframes = wrangle_data(cdhjson)

In [8]:
# Graph Plotting functions

def get_dataframes(df_names):
    dfs = [];
    for name in df_names:
        dfs.append(dataframes[name]);
    return dfs;

def make_key(plot):
    return plot.legend(
        title='Key',
        bbox_to_anchor=(0., 1.02, 1., .102),
        loc='lower left',
        ncol=1, 
        mode="expand", 
        borderaxespad=0.5,
        fontsize='xx-small'
    )

def plot_all_countrys(df_names, cols):
    """ Parameters: dataframes - list of dataframes, col - list of str dataframe column names for plotting
    Returns: single plot showing all countrys
    Plots all country data for a specified dataframe column on the same graph"""
    ax = None;
    count = 1;
    dataframes = get_dataframes(df_names);
    for df in dataframes:
        renamed_cols = {};
        for col in cols:
            if '_' in col:
                newcol = ((col.replace("_", " ")).replace('m', 'Males aged')).replace('f', 'Females aged');
                renamed_cols[col] = f"{df.iloc[0]['Country']}: {newcol}"
            else:
                renamed_cols[col] = f"{df.iloc[0]['Country']}: {col}"
        if count==1:
            ax = df[cols].rename(columns=renamed_cols).plot();
        else:
            ax = df[cols].rename(columns=renamed_cols).plot(ax=ax);
        count+=1;
    make_key(ax)
    ax.set_ylabel('Percentage of Population')
    return ax;

def plot_single_country(df_name, col):
    """ Parameters: dataframe - str name of dataframe, col - list of str dataframe column names for plotting
    Returns: single plot for one country
    Plots all country data for a specified dataframe column on the same graph"""
    renamed_cols = {};
    rename = False
    for name in col:
        if '_' in name:
            rename = True
            newcol = ((name.replace("_", " ")).replace('m', 'Males aged')).replace('f', 'Females aged');
            renamed_cols[name] = f"{dataframes[df_name].iloc[0]['Country']}: {newcol}"
    if rename:
        plt = dataframes[df_name][col].rename(columns=renamed_cols).plot();
    else:
        plt = dataframes[df_name][col].plot()
    make_key(plt)
    plt.set_ylabel('Percentage of Population')
    return plt

def show_grid():
    plt.grid(b=True, which='major', color='#666666', linestyle='-') # Show the major grid lines 
    plt.minorticks_on() # Show the minor grid lines
    plt.grid(b=True, which='minor', color='#777777', linestyle='-', alpha=0.2)
    return plt.show()

def plot_data(walk, graph):
    if type(walk) is list:
        plot_all_countrys(walk, graph)
    else:
        plot_single_country(walk, graph);
        show_grid();
    
def plot_age_data(walk, age, sex):
    plot_name = [];
    for age_group in age:
        for key in sex:
            plot_name.append(f'{key}_{age_group}')
    plot_data(walk, plot_name);
        
def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current = whichwalk.value
    if current == whichwalk.options['Scotland']:
        other = whichwalk.options['England']
    else:
        other = whichwalk.options['England']
    whichwalk.value = other # forces the redraw
    whichwalk.value = current # now we can change it back
    
    agecurrent = whichcountry_age.value
    if agecurrent == whichcountry_age.options['Scotland']:
        ageother = whichcountry_age.options['England']
    else:
        ageother = whichcountry_age.options['England']
    whichcountry_age.value = ageother # forces the redraw
    whichcountry_age.value = agecurrent # now we can change it back
    
def output_graph(graph):
    out = wdg.Output()
    with out:
        display(graph)
    return out

In [9]:
# API callback functions

def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = ['areaType=nation']

    # values here are the names of the PHE metrics
    structure = {
        "date":"date",
        "name":"areaName",
        "code":"areaCode",
        "cases": {
            "daily":"newCasesByPublishDate",
            "cumulative":"cumCasesByPublishDate",
            "males": "maleCases",
            "females": "femaleCases"
        },
        "deaths": {
            "daily":"newDeathsByDeathDate",
            "cumulative":"cumDeathsByDeathDate"
        },
        "hospitalisations": {
            "daily":"newAdmissions",
            "cumulative":"cumAdmissions",
            "cumulativebyage": "cumAdmissionsByAge"
        }
    }
    
    api = Cov19API(filters=filters, structure=structure)
    data = api.get_json()

    return data # return data read from the API

def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    
    apidata = access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global dataframes
    dataframes = wrangle_data(apidata)
    refresh_graph()
    apibutton.icon = "check"
    apibutton.description = "Data Updated"
    apibutton.disabled=True
    datebutton.description = f'Last Updated: {time.ctime(time.time())}'
    
main_layout = wdg.Layout(
    margin='0 auto',
    width = '100%'
)


<div class="alert alert-block alert-warning">The data currently displayed on this spreadsheet is out of date, please use the button bellow to update the data.</div>

In [10]:
apibutton = wdg.Button(
    description = 'Refresh Data',
    disabled = False,
    button_style = 'success',
    tooltip = "Update graphs with latest data",
    icon = 'download',
    layout=wdg.Layout(flex='1 1 auto', width='auto'),
)

apibutton.on_click(api_button_callback)

datebutton = wdg.Button(
    description = f'Last Updated: {time.ctime(os.path.getmtime("caseDeathHospitalisations.json"))}',
    disabled = True,
    layout=wdg.Layout(flex='4 1 auto', width='auto'),
)

items_auto = [apibutton, datebutton]

box_layout = wdg.Layout(
    display='flex',
    flex_flow='row',
    align_self='center',
    align_items='stretch',
    width = '100%'
)
refresh_box_auto = wdg.Box(children=items_auto, layout=box_layout)
refresh_data = wdg.VBox([refresh_box_auto])

refresh_data

## Cumulative Cases, Hospitalisations and Deaths Per Country as a Percentage of the Population of Each Country

The graph below shows an interactive plot where the view field can be used to switch between cumulative COVID-19 cases, hospitalisations and deaths. The country field can be used to compare data between countries or view single country data.

*N.B. Cumulative data on deaths is only available for England, all other countries will show 0% representing no data available* 

In [11]:
whichgraph = wdg.Dropdown(
    options = {
        'Cumulative Cases' : ['Cumulative Cases % of Population'],
        'Cumulative Hospitalisations' : ['Cumulative Hospitalisations % of Population'],
        'Cumulative Deaths' : ['Cumulative Deaths % of Population'],
    },
    value = ['Cumulative Cases % of Population'],
    description = 'View: ',
    disabled = False,
)
    
whichwalk = wdg.Dropdown(
    options = {
        'All':countries,
        'England':'England',
        'Northern Ireland':'Northern Ireland',
        'Scotland':'Scotland',
        'Wales':'Wales', 
    },
    value = countries,
    description = 'Country: ',
    disabled = False,
)   
    
cumulative_data_graph = wdg.interactive_output(
    plot_data,
    {'walk': whichwalk, 'graph': whichgraph}
)

#Cases, hospitalisations and deaths section
case_hosp_death_graph = output_graph(cumulative_data_graph);
case_hosp_death_widg = wdg.VBox([whichgraph, whichwalk]);
case_hosp_death_section = wdg.HBox([case_hosp_death_widg, case_hosp_death_graph], layout=box_layout);

case_hosp_death_section

## Cumulative Cases, by Age and Gender as a Percentage of the Population of that Age Group in Each Country


The graph below shows an interactive plot where the dropdown and select fields can be used to switch between cumulative COVID-19 cases, between males and females by their ages as a percentage of the population of that age group in each country. The country field can be used to compare data between countries or view single country data.

*N.B. Currently data is only available for England, all other countries will show 0% representing no data available. However, after updating the data using the button above data may be available for other countries* 

In [12]:
which_age = wdg.SelectMultiple(
    options=dict(zip(text_ages, ages)),
    value=['90+'],
    rows=2,
    description='Age Range:',
    disabled=False
)

whichcountry_age = wdg.Dropdown(
    options = {
        'All':['England_ages','Northern Ireland_ages','Scotland_ages','Wales_ages'],
        'England':'England_ages',
        'Northern Ireland':'Northern Ireland_ages',
        'Scotland': 'Scotland_ages',
        'Wales': 'Wales_ages', 
    },
    value = 'England_ages',
    description = 'Country: ',
    disabled = False,
)

which_sex = wdg.Dropdown(
    options = {
        'All': m_f,
        'Females': ['f'],
        'Males': ['m']
    },
    value = m_f,
    description = 'Gender: ',
    disabled = False,
)

cumulative_cases_age_graph = wdg.interactive_output(
    plot_age_data, 
    {'walk': whichcountry_age,
     'age': which_age,
     'sex': which_sex}
)
    
# display(whichcountry_age, which_sex, which_age, cumulative_cases_age_graph)
#Age section
age_graph = output_graph(cumulative_cases_age_graph);
age_widg = wdg.VBox([whichcountry_age, which_sex, which_age]);
age_section = wdg.HBox([age_widg, age_graph], layout=box_layout);

age_section